## PReader (old)
I couldn't make the PReader class be just a recursive algorithm. For long strings, this would mean that the stack would get very big. Some sort of buffer would have to prevent the stack from getting too big while also keeping the right order of characters. 

I created a char array from the string and a cursor. The cursor is a position indicator for which character is being read. In addition we would have to check for certain trigger characters, like operators for starting a new operation, parentheses to capture the operation in, and letters that would be variables. Taking this into account, we should also create a stack for read operations up until the current character. We would then interpret the operator when a closing parenthesis occurs.

## Testing arena below

In [1]:
from proposition import *
from proposition_parsing import *
from proposition_export import *
from hashing import *
from truthtable import *
from debugger import *

prop = OrProposition(OrProposition(Variable('A'), Variable('B')),
    Variable('C')
)

print(prop)
print(prop.infix())

|(|(A,B),C)
((A ∨ B) ∨ C)


In [2]:
truthtable = TruthTable(prop)

print(truthtable)

A B C result
0 0 0 0
0 0 1 1
0 1 0 1
0 1 1 1
1 0 0 1
1 0 1 1
1 1 0 1
1 1 1 1



In [3]:
simplified_truthtable = truthtable.simplify()

print(simplified_truthtable)

A B C result
0 0 0 0
* * 1 1
* 1 * 1
1 * * 1



In [4]:
print(simplified_truthtable.get_binary_string())
print(simplified_truthtable.get_hash())

1110
E


In [5]:
prop = PropositionParser("&(~(A),>(>(C,|(D,E)),~(B)))").read()
print(prop.infix())

truthtable = TruthTable(prop)
print(truthtable)

(¬A ∧ ((C => (D ∨ E)) => ¬B))
A B C D E result
0 0 0 0 0 1
0 0 0 0 1 1
0 0 0 1 0 1
0 0 0 1 1 1
0 0 1 0 0 1
0 0 1 0 1 1
0 0 1 1 0 1
0 0 1 1 1 1
0 1 0 0 0 0
0 1 0 0 1 0
0 1 0 1 0 0
0 1 0 1 1 0
0 1 1 0 0 1
0 1 1 0 1 0
0 1 1 1 0 0
0 1 1 1 1 0
1 0 0 0 0 0
1 0 0 0 1 0
1 0 0 1 0 0
1 0 0 1 1 0
1 0 1 0 0 0
1 0 1 0 1 0
1 0 1 1 0 0
1 0 1 1 1 0
1 1 0 0 0 0
1 1 0 0 1 0
1 1 0 1 0 0
1 1 0 1 1 0
1 1 1 0 0 0
1 1 1 0 1 0
1 1 1 1 0 0
1 1 1 1 1 0



In [6]:
truthtable.simplify()
print(truthtable)
print(f"hash: {truthtable.get_hash()}")

A B C D E result
0 0 0 0 0 1
0 0 0 0 1 1
0 0 0 1 0 1
0 0 0 1 1 1
0 0 1 0 0 1
0 0 1 0 1 1
0 0 1 1 0 1
0 0 1 1 1 1
0 1 0 0 0 0
0 1 0 0 1 0
0 1 0 1 0 0
0 1 0 1 1 0
0 1 1 0 0 1
0 1 1 0 1 0
0 1 1 1 0 0
0 1 1 1 1 0
1 * * * * 0

hash: 010FF


In [7]:
dnf_prop = truthtable.dnf()
print(dnf_prop.infix())

dnf_truthtable = TruthTable(dnf_prop)
dnf_truthtable.simplify()
print(dnf_truthtable)
print(f"hash: {dnf_truthtable.get_hash()}")

((¬A ∧ ¬B ∧ ¬C ∧ ¬D ∧ ¬E) ∨ (¬A ∧ ¬B ∧ ¬C ∧ ¬D ∧ E) ∨ (¬A ∧ ¬B ∧ ¬C ∧ D ∧ ¬E) ∨ (¬A ∧ ¬B ∧ ¬C ∧ D ∧ E) ∨ (¬A ∧ ¬B ∧ C ∧ ¬D ∧ ¬E) ∨ (¬A ∧ ¬B ∧ C ∧ ¬D ∧ E) ∨ (¬A ∧ ¬B ∧ C ∧ D ∧ ¬E) ∨ (¬A ∧ ¬B ∧ C ∧ D ∧ E) ∨ (¬A ∧ B ∧ C ∧ ¬D ∧ ¬E))
A B C D E result
0 0 0 0 0 1
0 0 0 0 1 1
0 0 0 1 0 1
0 0 0 1 1 1
0 0 1 0 0 1
0 0 1 0 1 1
0 0 1 1 0 1
0 0 1 1 1 1
0 1 0 0 0 0
0 1 0 0 1 0
0 1 0 1 0 0
0 1 0 1 1 0
0 1 1 0 0 1
0 1 1 0 1 0
0 1 1 1 0 0
0 1 1 1 1 0
1 * * * * 0

hash: 010FF


# Bigger test cases

In [8]:
def print_details(prop1, prop2):
    tt1 = TruthTable(prop1)
    tt2 = TruthTable(prop2)

    print(prop1.infix())
    print(prop1.ascii())
    print(tt1)
    print("hash: " + tt1.get_hash())
    print(prop2.infix())
    print(prop2.ascii())
    print(tt2)
    print("hash: " + tt2.get_hash())

In [9]:
# prop = PropositionParser("~(~(=(~(~(&(A,C))),B)))").read()
prop = PropositionParser("=(&(A,C),B)").read()

tfunc = lambda x : print(f"{x}")
analysis_hash = lambda prop : print(f"analysis - hash: {TruthTable(prop).get_hash()} - infix: {prop.infix()} - ascii: {prop.ascii_complex()}")
analyzing_functions = [analysis_hash]
my_debugger = Debugger(
        trace_function=tfunc, 
        analyzing_functions=analyzing_functions
    )

debugger_only_hash = Debugger(
        trace_function=lambda x : x, 
        analyzing_functions=[lambda p : print(f"hash: {TruthTable(p).get_hash()}")]
    )

prop_cnf = prop.cnf(
    debugger=my_debugger)
# print_details(prop, prop_cnf) 

start with: =(&(A,C),B)
analysis - hash: 93 - infix: ((A ∧ C) <=> B) - ascii: =(&(A,C),B)
removed biimpl: |(&(&(A,C),B),&(~(&(A,C)),~(B)))
analysis - hash: 93 - infix: (((A ∧ C) ∧ B) ∨ (¬(A ∧ C) ∧ ¬B)) - ascii: |(&(&(A,C),B),&(~(&(A,C)),~(B)))
removed impl: |(&(&(A,C),B),&(~(&(A,C)),~(B)))
analysis - hash: 93 - infix: (((A ∧ C) ∧ B) ∨ (¬(A ∧ C) ∧ ¬B)) - ascii: |(&(&(A,C),B),&(~(&(A,C)),~(B)))
removed nots: |(&(&(A,C),B),&(|(~(A),~(C)),~(B)))
analysis - hash: 93 - infix: (((A ∧ C) ∧ B) ∨ ((¬A ∨ ¬C) ∧ ¬B)) - ascii: |(&(&(A,C),B),&(|(~(A),~(C)),~(B)))
after distribution: *&([|(A,|(~(A),~(C))),|(A,~(B)),|(C,|(~(A),~(C))),|(C,~(B)),|(B,|(~(A),~(C))),|(B,~(B))])
analysis - hash: 93 - infix: ((A ∨ (¬A ∨ ¬C)) ∧ (A ∨ ¬B) ∧ (C ∨ (¬A ∨ ¬C)) ∧ (C ∨ ¬B) ∧ (B ∨ (¬A ∨ ¬C)) ∧ (B ∨ ¬B)) - ascii: *&([|(A,|(~(A),~(C))),|(A,~(B)),|(C,|(~(A),~(C))),|(C,~(B)),|(B,|(~(A),~(C))),|(B,~(B))])
merged ors: *&([*|([A,~(A),~(C)]),*|([A,~(B)]),*|([C,~(A),~(C)]),*|([C,~(B)]),*|([B,~(A),~(C)]),*|([B,~(B)])])
analysis 

In [10]:
# print_details(prop, PropositionParser("|(&(&(A,C),B),&(~(&(A,C)),~(B)))").read())

In [11]:
from equality_tester import *
# prop = PropositionParser("=(&(A,C),B)").read()
prop = PropositionParser("=(>(A,~(D)),&(~(~(B)),~(|(C,D))))").read()
# prop = PropositionParser("&(E,=(A,|(&(B,C),&(D,~(C)))))").read()
print(f"Expected hash: {TruthTable(prop).get_hash()}")
prop_cnf = prop.cnf(
    debugger=my_debugger)

# print(TruthTable(prop))

# print(f"cnf notation: {prop_cnf.cnf_notation()}")

Expected hash: BA10
start with: =(>(A,~(D)),&(~(~(B)),~(|(C,D))))
analysis - hash: BA10 - infix: ((A => ¬D) <=> (¬¬B ∧ ¬(C ∨ D))) - ascii: =(>(A,~(D)),&(~(~(B)),~(|(C,D))))
removed biimpl: |(&(>(A,~(D)),&(~(~(B)),~(|(C,D)))),&(~(>(A,~(D))),~(&(~(~(B)),~(|(C,D))))))
analysis - hash: BA10 - infix: (((A => ¬D) ∧ (¬¬B ∧ ¬(C ∨ D))) ∨ (¬(A => ¬D) ∧ ¬(¬¬B ∧ ¬(C ∨ D)))) - ascii: |(&(>(A,~(D)),&(~(~(B)),~(|(C,D)))),&(~(>(A,~(D))),~(&(~(~(B)),~(|(C,D))))))
removed impl: |(&(|(~(A),~(D)),&(~(~(B)),~(|(C,D)))),&(~(|(~(A),~(D))),~(&(~(~(B)),~(|(C,D))))))
analysis - hash: BA10 - infix: (((¬A ∨ ¬D) ∧ (¬¬B ∧ ¬(C ∨ D))) ∨ (¬(¬A ∨ ¬D) ∧ ¬(¬¬B ∧ ¬(C ∨ D)))) - ascii: |(&(|(~(A),~(D)),&(~(~(B)),~(|(C,D)))),&(~(|(~(A),~(D))),~(&(~(~(B)),~(|(C,D))))))
removed nots: |(&(|(~(A),~(D)),&(B,&(~(C),~(D)))),&(&(A,D),|(~(B),|(C,D))))
analysis - hash: BA10 - infix: (((¬A ∨ ¬D) ∧ (B ∧ (¬C ∧ ¬D))) ∨ ((A ∧ D) ∧ (¬B ∨ (C ∨ D)))) - ascii: |(&(|(~(A),~(D)),&(B,&(~(C),~(D)))),&(&(A,D),|(~(B),|(C,D))))
after distribution: *&

In [12]:
prop = PropositionCNFFormatParser("[ E , Abc , AdC , BDa , Bca , CDa ]").read()
print(prop.infix())
print(f"Hash: {TruthTable(prop).get_hash()}")

(E ∧ (A ∨ ¬B ∨ ¬C) ∧ (A ∨ ¬D ∨ C) ∧ (B ∨ D ∨ ¬A) ∧ (B ∨ ¬C ∨ ¬A) ∧ (C ∨ D ∨ ¬A))
Hash: A80802A2


# Tseitin

In [13]:
prop = PropositionParser("=(&(~(s),p),|(>(q,r),~(p)))").read()
print(f"original : {prop.infix()}")

tseitin = prop.tseitin(debugger=my_debugger)
print(f"tseitin  : {tseitin.infix()}")

tseitin_cnf = tseitin.cnf()
print(f"cnf'ed   : {tseitin_cnf.infix()}")
# print(f"cnf'ed   : {tseitin_cnf.ascii_complex()}")

original : ((¬s ∧ p) <=> ((q => r) ∨ ¬p))
tseitin start with: =(&(~(s),p),|(>(q,r),~(p)))
analysis - hash: 6500 - infix: ((¬s ∧ p) <=> ((q => r) ∨ ¬p)) - ascii: =(&(~(s),p),|(>(q,r),~(p)))
after tseitin repl: *&([A,=(A,=(B,D)),=(B,&(C,p)),=(C,~(s)),=(D,|(E,F)),=(E,>(q,r)),=(F,~(p))])
analysis - hash: 0000450000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000200000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000 - infix: (A ∧ (A <=> (B <=> D)) ∧ (B <=> (C ∧ p)) ∧ (C <=> ¬s) ∧ (D <=> (E ∨ F)) ∧ (E <=> (q => r)) ∧ (F <=> ¬p)) - ascii: *&([A,=(A,=(B,D)),=(B,&(C,p)),=(C,~(s)),=(D,|(E,F)),=(E,>(q,r)),=(F,~(p))])
tseitin  : (A ∧ (A <=> (B <=> D)) ∧ (B <=> (C ∧ p)) ∧ (C <=> ¬s) ∧ (D <=> (E ∨ F)) ∧ (E <=> (q => r)) ∧ (F <=> ¬p))
cnf'ed   : (A ∧ (A ∨ ¬B ∨ ¬D) ∧ (A ∨ B ∨ D) ∧ (B ∨ ¬D ∨ ¬A) ∧ (D ∨ ¬B ∨ ¬A) ∧ (B ∨ ¬C ∨ ¬p) ∧ (C ∨ ¬B) ∧ (p ∨ ¬B) ∧ (C ∨ s) ∧ 

In [15]:
# Joris testcase 1
prop = PropositionParser("=(A,~(&(B,C)))").read()
print(f"original : {prop.infix()}")

tseitin = prop.tseitin(debugger=my_debugger)
print(f"tseitin  : {tseitin.infix()}")

tseitin_cnf = tseitin.cnf()
print(f"cnf'ed   : {tseitin_cnf.infix()}")
print(f"cnf'ed   : {tseitin_cnf.cnf_notation()}")

original : (A <=> ¬(B ∧ C))
tseitin start with: =(A,~(&(B,C)))
analysis - hash: 78 - infix: (A <=> ¬(B ∧ C)) - ascii: =(A,~(&(B,C)))
after tseitin repl: *&([D,=(D,=(A,E)),=(E,~(F)),=(F,&(B,C))])
analysis - hash: 0040404020000000 - infix: (D ∧ (D <=> (A <=> E)) ∧ (E <=> ¬F) ∧ (F <=> (B ∧ C))) - ascii: *&([D,=(D,=(A,E)),=(E,~(F)),=(F,&(B,C))])
tseitin  : (D ∧ (D <=> (A <=> E)) ∧ (E <=> ¬F) ∧ (F <=> (B ∧ C)))
cnf'ed   : (D ∧ (D ∨ ¬A ∨ ¬E) ∧ (D ∨ A ∨ E) ∧ (A ∨ ¬E ∨ ¬D) ∧ (E ∨ ¬A ∨ ¬D) ∧ (E ∨ F) ∧ (¬F ∨ ¬E) ∧ (F ∨ ¬B ∨ ¬C) ∧ (B ∨ ¬F) ∧ (C ∨ ¬F))
cnf'ed   : [ D , Dae , DAE , Aed , Ead , EF , fe , Fbc , Bf , Cf ]


In [16]:
# Joris testcase 2
prop = PropositionParser("&(A,~(=(|(B,D),C)))").read()
print(f"original : {prop.infix()}")

tseitin = prop.tseitin(debugger=my_debugger)
print(f"tseitin  : {tseitin.infix()}")

tseitin_cnf = tseitin.cnf()
print(f"cnf'ed   : {tseitin_cnf.infix()}")
print(f"cnf'ed   : {tseitin_cnf.cnf_notation()}")

original : (A ∧ ¬((B ∨ D) <=> C))
tseitin start with: &(A,~(=(|(B,D),C)))
analysis - hash: 3600 - infix: (A ∧ ¬((B ∨ D) <=> C)) - ascii: &(A,~(=(|(B,D),C)))
after tseitin repl: *&([E,=(E,&(A,F)),=(F,~(G)),=(G,=(H,C)),=(H,|(B,D))])
analysis - hash: 0000000020002000000010002000000000000000000000000000000000000000 - infix: (E ∧ (E <=> (A ∧ F)) ∧ (F <=> ¬G) ∧ (G <=> (H <=> C)) ∧ (H <=> (B ∨ D))) - ascii: *&([E,=(E,&(A,F)),=(F,~(G)),=(G,=(H,C)),=(H,|(B,D))])
tseitin  : (E ∧ (E <=> (A ∧ F)) ∧ (F <=> ¬G) ∧ (G <=> (H <=> C)) ∧ (H <=> (B ∨ D)))
cnf'ed   : (E ∧ (E ∨ ¬A ∨ ¬F) ∧ (A ∨ ¬E) ∧ (F ∨ ¬E) ∧ (F ∨ G) ∧ (¬G ∨ ¬F) ∧ (G ∨ ¬H ∨ ¬C) ∧ (G ∨ H ∨ C) ∧ (H ∨ ¬C ∨ ¬G) ∧ (C ∨ ¬H ∨ ¬G) ∧ (H ∨ ¬B) ∧ (H ∨ ¬D) ∧ (B ∨ D ∨ ¬H))
cnf'ed   : [ E , Eaf , Ae , Fe , FG , gf , Ghc , GHC , Hcg , Chg , Hb , Hd , BDh ]
